In [ ]:
from pysis import isis

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np
import math

import matplotlib.pyplot as plt
import pandas as pd
from knoten.bundle import *

pd.options.mode.copy_on_write = True
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
ale.spice_root='~/data'

In [3]:
cubes = 'data/original-dataset/cubes.lis'
sensors = generate_sensors(cubes)

network = 'data/original-dataset/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
sensors = {sn: sensors[sn] for sn in cnet["serialnumber"].unique()}
cnet = compute_apriori_ground_points(cnet, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

MRO/CTX/1085197697:073
MRO/CTX/1096561308:045
MRO/CTX/1136952576:186
MRO/CTX/1157902986:250


In [14]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}

In [13]:
# Solve for angles and angular rates
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

In [6]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)

In [7]:
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)
dof = W_observations.shape[0] - W_params.shape[0]

In [8]:
max_iterations = 10
tol = 1e-10

In [9]:
num_sensor_params = sum([len(parameters) for parameters in solve_parameters.values()])
num_point_params = 3 * len(cnet["id"].unique())

In [10]:
def get_outlier_mask(V, threshold=5):
    """ Create a list of euclidean distances sqrt((x2-x1)^2 + (y2-y1)^2) where (x1,y1) is (0,0) i.e. "no error"
    
    Params
    ------
    V : np.array
        Residuals in the form [line_residual1, sample_residual1, line_residual2, sample_residual2 ...]
        
    threshold : int
        The maximum allowable distance (in pixels) to be considered in inlier

    """
    # Coerce residuals data into Nx2 (line, sample pairs)
    point_residuals = V.reshape(-1,2)
    return [math.sqrt((sr**2) + (lr**2)) < threshold for sr,lr in point_residuals]

# Notes

- Upper case generally denotes matrix notation, lower case typically denotes vector notation
- In this case, the Hessian may be the transpose Jacobian, i.e.  𝐇(𝑓(𝐱))=𝐉(∇𝑓(𝐱))𝑇 .  Unclear if this holds for sparse matrix. Notes from Jesse state "Inverse of components of Hessian != inverse of the Hessian"
- The Hessian is modeled/denoted as follows:
    ``` 
        -------------------------
       |            |            |
       |            |            |
       | H_CC (N11) | H_CP (N12) |
       |            |            |
       |            |            |
       |------------+------------|
       |            |            |
       |            |            |
       | H_PC (N21) | H_PP (N22) |
       |            |            |
       |            |            |
        -------------------------
    ```
- Isis/CSM use a mix of H notation and N notation.  C is cameras/sensors, P is points, so H_CC is camera*camera, etc.
- H_PC and H_CP are symmetric, so we only want to calculate one.  In this way, we compute a "triangular matrix" by ignoring upper right or lower left.
- NIC vectors described [here](https://lear.inrialpes.fr/people/triggs/pubs/Brown-IAP76.pdf).  Unclear what they are used for. _seems_ to be H_PP * corrected ground points
- g_P seems to be n2 in ISIS, described as "The right hand side vector for the point on the body." (BundleAdjust.cpp)
- g_C seems to be nj in ISIS, described as "The output right hand side vector." (BundleAdjust.cpp)

## Omega_i:
- $ \omega_i $ is the test statistic used to determine whether or not a point is an outlier.
- In the context of bundle adustment, $ \omega_i $ is calculated as $ \omega_i = -v_i / \sigma_{v_i} $
- $ \sigma_i = \sigma_0 \sqrt{Q_{v_i v_i}} $
- $ \sqrt{Q_{v_i v_i}} $ is the diagonal of $ Q_{vv} $
- $ Q_{vv} = Q_{ll} - A {(A^T P_{ll} A)}^{-1} A^T $
- $ Q_{ll} $ is the observation weights
- $ {(A^T P_{ll} A)}^{-1} $ is $ N^{-1} $
- $ A $ is the jacobian / design matrix


-  More information can be found [here](https://www.asprs.org/wp-content/uploads/pers/1985journal/aug/1985_aug_1137-1149.pdf)

- Most of this information is available in reduced normal equation, but A is not.
- May be possible to calculate standardized residuals from sparse matrix using [this](https://stackoverflow.com/questions/12305021/efficient-way-to-normalize-a-scipy-sparse-matrix) method.


In [11]:
max_iterations = 10
sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

# This is setup once per bundle and then accumulates
total_corrections = np.zeros(num_sensor_params + num_point_params)

# These are computed once per bundle
# W_CC is sensor*sensor weights
W_CC_sparse = sparse.csc_matrix((num_sensor_params, num_sensor_params))
# W_PP is point*point weights
W_PP = {}

# Compute image param weight matrices
for sn, params in solve_parameters.items():
    coeff_indices = column_dict[sn]
    coeff_range = np.arange(coeff_indices[0], coeff_indices[1])
    num_image_coeffs = coeff_indices[1] - coeff_indices[0]
    W_CC_data = compute_image_weight(sensors[sn], params).ravel()
    W_CC_row = np.repeat(coeff_range, num_image_coeffs)
    W_CC_column = np.tile(coeff_range, num_image_coeffs)
    W_CC_sparse += sparse.coo_matrix((W_CC_data, (W_CC_row, W_CC_column)), (num_sensor_params, num_sensor_params)).tocsc()
    
# Compute point param weight matrices
for point_id in cnet['id'].unique():
    W_PP[point_id] = compute_point_weight(cnet, point_id)

# Set all mask values True
cnet['mask'] = True
V = compute_residuals(cnet, sensors)
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
sigma0 = compute_sigma0_sparse(V, np.zeros(total_corrections.shape), W_CC_sparse, W_PP, W_observations, column_dict)

# Begin bundle adjustment process
for i in range(max_iterations):
    # Only consider non-rejected points
    num_observations = 2 * len(cnet[cnet['mask']==True])
    # Weight all observations the same
    W_observations = np.eye(num_observations)
    
    # Keep track of previous sigma to check for convergence criterion
    old_sigma0 = sigma0
    
    # Set up sparse camera x camera Hessian (H11 / Upper Left Hessian)
    H_CC_sparse = sparse.csc_matrix((num_sensor_params, num_sensor_params))
    H_CC_sparse += W_CC_sparse

    # gradient of cost function for Camera (sensor) parameters
    g_C_sparse = np.zeros(num_sensor_params)
    g_C_sparse -= W_CC_sparse.dot(total_corrections[:num_sensor_params])
    # g_C_sparse += W_CC_sparse.dot(sensor_corrections)

    #Q = H_PP^-1 * H_PC 
    Q_mats = {}
    # NIC = H_PP^-1 * g_P
    #   g_P is "right hand side of normal equation." Gradient of cost function w.r.t. points?
    #   "Normalized Image Coordinates"
    NIC_vecs = {}

    updates = np.zeros(num_sensor_params + num_point_params)

    # For each ground point
    for point_id, group in cnet.groupby('id'):
        ground_pt = group.iloc[0][["adjustedX", "adjustedY", "adjustedZ"]]
        
        # Hessian segment for camera * point, n_rows = n_sensor params, n_cols = 3 (x,y,z coord)
        H_CP = sparse.csc_matrix((num_sensor_params, 3))
        # Hessian segment for point * point, 3x3 for (x,y,z ; x,y,z)
        H_PP = np.zeros((3, 3))
        # Corrected ground points (x,y,z)
        g_P = np.zeros(3)
        
        # For each measure associated with the ground point
        for vi, (measure_idx, row) in enumerate(group.iterrows()):
            # Do not use any measure that is masked out
            if row['mask'] == False:
                continue
            serial = row["serialnumber"]
            sensor = sensors[serial]
            
            # First order partial derivatives (Jacobian)
            point_partials = compute_ground_partials(sensor, ground_pt)
            sensor_partials = compute_sensor_partials(sensor, solve_parameters[serial], ground_pt)
            coeff_indices = column_dict[serial]
            coeff_range = np.arange(coeff_indices[0], coeff_indices[1])
            num_image_coeffs = coeff_indices[1] - coeff_indices[0]
            
            # Construct camera to camera Hessian segment (n_params x n_params 1 dimensional)
            H_CC_point_data = np.dot(sensor_partials.T, sensor_partials).ravel()
            H_CC_point_row = np.repeat(coeff_range, num_image_coeffs)
            H_CC_point_column = np.tile(coeff_range, num_image_coeffs)
            H_CC_sparse += sparse.coo_matrix((H_CC_point_data, (H_CC_point_row, H_CC_point_column)), (num_sensor_params, num_sensor_params)).tocsc()

            # Construct a camera to point Hessian segment (n_params x 3 (x,y,z), 1 dimensional)
            H_CP_point_data = np.dot(sensor_partials.T, point_partials).ravel()
            H_CP_point_row = np.repeat(coeff_range, 3)
            H_CP_point_column = np.tile(np.arange(0, 3), num_image_coeffs)
            H_CP += sparse.coo_matrix((H_CP_point_data, (H_CP_point_row, H_CP_point_column)), (num_sensor_params, 3)).tocsc()

            # Construct point to point Hessian segment
            H_PP += np.dot(point_partials.T, point_partials)

            # Sparse gradient of cost function with respect to 
            # Note: Use 'vi' instead of measure_idx because measure_idx does not correspond 1:1 with V.
            g_C_sparse[coeff_indices[0]:coeff_indices[1]] += np.dot(sensor_partials.T, V[2*vi:2*vi+2])
            g_P += np.dot(point_partials.T, V[2*vi:2*vi+2])
        point_param_range = column_dict[point_id]
        # Gradient of cost function with respect to points?
        g_P -= W_PP[point_id].dot(total_corrections[point_param_range[0]:point_param_range[1]])
        
        # This is just adding the point weight to the Hessian??? Should this be *= ?
        H_PP += W_PP[point_id]
        H_PP_inv = sparse.csc_matrix(np.linalg.inv(H_PP))
        
        Q_mats[point_id] = H_PP_inv.dot(H_CP.transpose())
        
        # NIC = Point*Point Hessian * gradient of cost w.r.t. points
        NIC_vecs[point_id] = H_PP_inv.dot(g_P)
        
        H_CC_sparse -= H_CP.dot(Q_mats[point_id])
        # Gradient of cost function w.r.t. camera parameters?
        g_C_sparse -= H_CP.dot(NIC_vecs[point_id])

    # Updates related to camera parameters. solve for (Normal equation = Gradient of Camera Parameters)?
    updates[:num_sensor_params] = sparse.linalg.spsolve(H_CC_sparse, g_C_sparse)

    # Incorporate point information into updates
    for point_id in Q_mats:
        point_param_indices = column_dict[point_id]
        # Point updates = Normalized Image Coordinates - Q_Matrix . sensor updates ?
        updates[point_param_indices[0]:point_param_indices[1]] = NIC_vecs[point_id] - Q_mats[point_id].dot(updates[:num_sensor_params])
        
    print(f'corrections: mean = {updates.mean()} min = {updates.min()} max = {updates.max()}')

    total_corrections += updates
    update_parameters(sensors, solve_parameters, cnet[cnet['mask']==True], updates, column_dict)

    V = compute_residuals(cnet[cnet['mask']==True], sensors)

    sigma0 = compute_sigma0_sparse(V, updates, W_CC_sparse, W_PP, W_observations, column_dict)
    print(f'iteration {i+1}: sigma0 = {sigma0}\n')
    
    # Write residuals back to dataframe. Shape/order of V matches masked cnet
    cnet.loc[cnet['mask']==True, ['lineResidual','sampleResidual']] = V.reshape(-1,2)
    # Mask new values
    cnet.loc[cnet['mask']==True, 'mask'] = get_outlier_mask(V, 7.5)
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break

MRO/CTX/1085197697:073
MRO/CTX/1096561308:045
MRO/CTX/1136952576:186
MRO/CTX/1157902986:250
num measures: 130
corrections: mean = -0.0954163547457873 min = -11.784284890035664 max = 7.44137907853877
num measures: 130
iteration 1: sigma0 = 4.902019910572186

corrections: mean = -0.022491583525547122 min = -6.973923511478597 max = 5.0792773525762875
num measures: 127
iteration 2: sigma0 = 4.717912730649052

corrections: mean = -0.1415581908051954 min = -5.814579457774052 max = 3.0950151469804137
num measures: 121
iteration 3: sigma0 = 4.930134709649926

corrections: mean = -0.39184821611524084 min = -16.585766760663184 max = 5.477596884710749
num measures: 101
iteration 4: sigma0 = 4.455717534253432

corrections: mean = 0.18887092874352393 min = -3.6289063390576253 max = 15.555114495599623
num measures: 96
iteration 5: sigma0 = 4.101771821025744

corrections: mean = 0.04313070391050939 min = -3.3064299878256986 max = 5.66153400723288
num measures: 96
iteration 6: sigma0 = 3.9859083761098

In [12]:
cnet

,id,pointType,pointChoosername,pointDatetime,pointEditLock,pointIgnore,pointJigsawRejected,referenceIndex,aprioriSurfPointSource,aprioriSurfPointSourceFile,aprioriRadiusSource,aprioriRadiusSourceFile,latitudeConstrained,longitudeConstrained,radiusConstrained,aprioriX,aprioriY,aprioriZ,aprioriCovar,adjustedX,adjustedY,adjustedZ,adjustedCovar,pointLog,serialnumber,measureType,sample,line,sampleResidual,lineResidual,measureChoosername,measureDatetime,measureEditLock,measureIgnore,measureJigsawRejected,diameter,apriorisample,aprioriline,samplesigma,linesigma,measureLog,mask
0,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,0,,False,False,False,2919381.80,-1194122.57,1238053.92,"[4.735705191536794, -0.8861958818176927, 1.885...",2919381.80,-1194122.57,1238053.92,[],[],MRO/CTX/1085197697:073,0,200.86,302.07,-0.29,0.20,Unknown,2019-12-06T10:19:12,False,False,False,0.00,200.86,302.07,0.00,0.00,[],True
1,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,0,,False,False,False,2919381.80,-1194122.57,1238053.92,"[4.735705191536794, -0.8861958818176927, 1.885...",2919381.80,-1194122.57,1238053.92,[],[],MRO/CTX/1157902986:250,1,744.18,15166.79,0.19,-7.50,Unknown,2019-12-06T10:19:12,False,False,False,0.00,753.59,15167.63,0.00,0.00,[],False
2,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,0,,False,False,False,2919381.80,-1194122.57,1238053.92,"[4.735705191536794, -0.8861958818176927, 1.885...",2919381.80,-1194122.57,1238053.92,[],[],MRO/CTX/1096561308:045,1,406.44,13311.62,0.49,-0.20,Unknown,2019-12-06T10:19:12,False,False,False,0.00,399.44,13311.57,0.00,0.00,[],True
3,autoseed_002,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,0,,False,False,False,2923302.70,-1184955.39,1238123.26,"[4.55809400691766, -1.002692929635315, 1.81139...",2923302.70,-1184955.39,1238123.26,[],[],MRO/CTX/1085197697:073,0,1925.96,95.88,-0.54,0.05,Unknown,2019-12-06T10:19:12,False,False,False,0.00,1925.96,95.88,0.00,0.00,[],True
4,autoseed_002,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,0,,False,False,False,2923302.70,-1184955.39,1238123.26,"[4.55809400691766, -1.002692929635315, 1.81139...",2923302.70,-1184955.39,1238123.26,[],[],MRO/CTX/1157902986:250,1,2092.57,14937.45,0.25,-7.87,Unknown,2019-12-06T10:19:12,False,False,False,0.00,2099.73,14938.33,0.00,0.00,[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,hand_15,2,ladoramkershner,2019-12-11T16:05:47,False,False,False,0,0,,0,,False,False,False,2902436.84,-1181686.53,1289358.53,"[4.5271199192478555, -1.0324332221678945, 1.88...",2902436.84,-1181686.53,1289358.53,[],[],MRO/CTX/1096561308:045,1,2420.47,22463.22,1.22,-1.07,Unknown,2019-12-11T16:05:47,False,False,False,0.00,2413.42,22462.17,0.00,0.00,[],True
126,hand_15,2,ladoramkershner,2019-12-11T16:05:47,False,False,False,0,0,,0,,False,False,False,2902436.84,-1181686.53,1289358.53,"[4.5271199192478555, -1.0324332221678945, 1.88...",2902436.84,-1181686.53,1289358.53,[],[],MRO/CTX/1157902986:250,1,2439.21,24261.52,0.51,-7.81,Unknown,2019-12-11T16:05:47,False,False,False,0.00,2445.48,24263.40,0.00,0.00,[],False
127,hand_16,2,ladoramkershner,2019-12-12T08:09:54,False,False,False,1,0,,0,,False,False,False,2906426.36,-1172437.64,1289984.34,"[4.3566549912676305, -1.1427182025503582, 1.81...",2906426.36,-1172437.64,1289984.34,[],[],MRO/CTX/1157902986:250,1,3841.89,24113.85,0.51,-8.08,Unknown,2019-12-12T08:09:54,False,False,False,0.00,3843.42,24115.32,0.00,0.00,[],False
128,hand_16,2,ladoramkershner,2019-12-12T08:09:54,False,False,False,1,0,,0,,False,False,False,2906426.36,-1172437.64,1289984.34,"[4.3566549912676305, -1.1427182025503582, 1.81...",2906426.36,-1172437.64,1289984.34,[],[],MRO/CTX/1085197697:073,1,4009.14,9314.20,-0.62,-3.36,Unknown,2019-12-12T08:09:54,False,False,False,0.00,4009.14,9314.20,0.00,0.00,[],True
